In [2]:
import pandas as pd
import numpy as np

In [8]:
#filepath = 'https://raw.githubusercontent.com/jubins/ML-TwitterBotDetection/master/MidwayReport/data/'
filepath = 'https://raw.githubusercontent.com/jubins/ML-TwitterBotDetection/master/FinalCode/kaggle_data/'
df = pd.read_csv(filepath+'training_data_2_csv_UTF.csv')
bots = df[:1320]
nonbots = df[1321:]

In [9]:
#bots[bots.listedcount>10000]
condition = (bots.screen_name.str.contains("bot", case=False)==True)|(bots.description.str.contains("bot", case=False)==True)|(bots.location.isnull())|(bots.verified==False)

bots['screen_name_binary'] = (bots.screen_name.str.contains("bot", case=False)==True)
bots['description_binary'] = (bots.description.str.contains("bot", case=False)==True)
bots['location_binary'] = (bots.location.isnull())
bots['verified_binary'] = (bots.verified==False)
bots.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

(1320, 24)

In [10]:
condition = (nonbots.screen_name.str.contains("bot", case=False)==False)| (nonbots.description.str.contains("bot", case=False)==False) |(nonbots.location.isnull()==False)|(nonbots.verified==True)

nonbots['screen_name_binary'] = (nonbots.screen_name.str.contains("bot", case=False)==False)
nonbots['description_binary'] = (nonbots.description.str.contains("bot", case=False)==False)
nonbots['location_binary'] = (nonbots.location.isnull()==False)
nonbots['verified_binary'] = (nonbots.verified==True)
nonbots.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

(1476, 24)

In [11]:
df = pd.concat([bots, nonbots])

In [47]:
def vectorize_features(df):
    df['screen_name_binary'] = (df.screen_name.str.contains("bot", case=False)) #If it contains bot
    df['description_binary'] = (df.description.str.contains("bot", case=False))
    df['location_binary'] = (df.location.isnull())
    df['verified_binary'] = (df.verified==False)
    return df

df.screen_name.str.contains("bot", case=False)

0       False
1       False
2        True
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22       True
23       True
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
2767    False
2768    False
2769    False
2770    False
2771    False
2772    False
2773    False
2774    False
2775    False
2776    False
2777    False
2778    False
2779    False
2780    False
2781    False
2782    False
2783    False
2784    False
2785    False
2786    False
2787    False
2788    False
2789    False
2790    False
2791     True
2792    False
2793    False
2794    False
2795    False
2796    False
Name: screen_name, dtype: bool

### Using Decision Tree Classifier

In [35]:
from sklearn.model_selection import train_test_split
train_df = df
test_df = pd.read_csv('https://raw.githubusercontent.com/jubins/ML-TwitterBotDetection/master/FinalCode/kaggle_data/test_data_4_students.csv')
test_df = vectorize_features(test_df)
test_df.head(2)

,id,id_str,screen_name,location,description,url,followers_count,friends_count,listed_count,created_at,...,status,default_profile,default_profile_image,has_extended_profile,name,bot,screen_name_binary,description_binary,location_binary,verified_binary
0,2.281293e+09,2281292622,__keating,brooklyn,lgbt editor at @buzzfeed. shannon.keating@buzz...,https://t.co/QneJmYRyhj,4466,1295,111,Tue Jan 07 23:26:52 +0000 2014,...,{'created_at': 'Tue Apr 11 15:31:51 +0000 2017...,FALSE,FALSE,FALSE,shannon keating,NaN,False,False,False,False
1,2.344040e+09,2344040251,_callme_Dani,"Los Angeles, CA",News Curation Editor @BuzzFeedNews \rI do a lo...,NaN,295,1016,10,Fri Feb 14 19:45:56 +0000 2014,...,{'created_at': 'Tue Apr 11 00:56:02 +0000 2017...,FALSE,FALSE,FALSE,Danielle Hopkins,NaN,False,False,False,False


In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

clf = DecisionTreeClassifier(criterion='entropy')

#80%
X_train = train_df[['screen_name_binary', 'description_binary', 'location_binary', 'verified_binary']] #train_data 
y_train = train_df['bot'] #train_target

#20%
X_test = test_df[['screen_name_binary', 'description_binary', 'location_binary', 'verified_binary']] #test_Data
y_test = test_df['bot'] #test_target

#Training on decision tree classifier
model = clf.fit(X_train, y_train)

#Predicting on test data
#model.predict(X_test)

In [43]:
#Checking accuracy
#print("Decision Tree Classifier Accuracy: {0}".format(accuracy_score(y_test, predicted)))

,screen_name_binary,description_binary,location_binary,verified_binary
0,False,False,False,False
1,False,False,False,False
2,False,False,True,False
3,False,False,True,False
4,True,True,False,False
5,True,True,False,False
6,True,True,False,False
7,True,True,False,False
8,True,True,False,False
9,False,False,False,False


### Using Random Forest Classifier

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier(min_samples_split=50, min_samples_leaf=200)

#80%
X_train = train_df[['screen_name_binary', 'description_binary', 'location_binary', 'verified_binary']] #train_data 
y_train = train_df['bot'] #train_target

#20%
X_test = test_df[['screen_name_binary', 'description_binary', 'location_binary', 'verified_binary']] #test_Data
y_test = test_df['bot'] #test_target

#Training on decision tree classifier
model = clf.fit(X_train, y_train)

#Predicting on test data
predicted = model.predict(X_test)

#Checking accuracy
print("Random Forest Classifier Accuracy: {0}".format(accuracy_score(y_test, predicted)))

Random Forest Classifier Accuracy: 0.9373601789709173


### Using Multinomial Naive Bayes Classifier

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

clf = MultinomialNB(alpha=1, fit_prior=True)

#80%
X_train = train_df[['screen_name_binary', 'description_binary', 'location_binary', 'verified_binary']] #train_data 
y_train = train_df['bot'] #train_target

#20%
X_test = test_df[['screen_name_binary', 'description_binary', 'location_binary', 'verified_binary']] #test_Data
y_test = test_df['bot'] #test_target

#Training on decision tree classifier
model = clf.fit(X_train, y_train)

#Predicting on test data
predicted = model.predict(X_test)

#Checking accuracy
print("MultinomialNB Classifier Accuracy: {0}".format(accuracy_score(y_test, predicted)))

MultinomialNB Classifier Accuracy: 0.9373601789709173
